In [1]:
!pip -q install sentence-transformers scikit-learn numpy


In [2]:
import os, json, math, re
import numpy as np
from typing import Dict, List, Tuple
from dataclasses import dataclass
from sentence_transformers import SentenceTransformer, util

# ====== (A) 라벨 사전: 현장 용어/별칭을 자유롭게 추가하세요 ======
FAULT_PARTS: Dict[str, List[str]] = {
    "유니온": ["유니온", "에어 유니온", "50A 유니온", "공압 유니온", "피팅"],
    "밸브": ["밸브", "솔레노이드 밸브", "에어밸브", "조절밸브"],
    "호스": ["호스", "에어호스", "튜브"],
    "실린더": ["실린더", "공압 실린더", "에어 실린더"],
    "모터": ["모터", "서보모터"],
    "베어링": ["베어링"],
    "펌프": ["펌프"],
    "센서": ["센서", "포토센서", "근접센서", "리미트 스위치"],
    "노즐": ["노즐", "분사노즐"],
    "컨베이어": ["컨베이어", "벨트", "라인벨트"]
}

DEFECT_TYPES: Dict[str, List[str]] = {
    "에어리크": ["에어리크", "에어 리크", "공기누설", "누기", "누설"],
    "파손": ["파손", "깨짐", "크랙"],
    "마모": ["마모", "닳음"],
    "오염": ["오염", "이물", "때"],
    "정렬불량": ["정렬불량", "얼라인 불량", "편심", "치우침"],
    "전기불량": ["전기불량", "단선", "합선", "센서불량"],
    "간극과다": ["간극", "유격", "클리어런스 과다"],
    "과열": ["과열", "온도상승", "과온"],
    "진동": ["진동", "떨림", "소음 증가"]
}

ACTIONS: Dict[str, List[str]] = {
    "재조임": ["재조임", "볼트 조임", "체결", "재체결", "토크 재설정"],
    "교체": ["교체", "교환", "부품 갈음"],
    "청소": ["청소", "세척", "클리닝"],
    "윤활": ["윤활", "그리스 주입", "오일링"],
    "재정렬": ["재정렬", "얼라인 조정", "정렬"],
    "수리": ["수리", "보수", "보강"],
    "점검": ["점검", "확인", "체크"]
}

# ====== (B) 임베딩 모델 로드 ======
model_id = "jhgan/ko-sroberta-multitask"  # 한국어 SBERT, CPU에서도 빠름
encoder = SentenceTransformer(model_id)    # from_pretrained 경로에 로컬 폴더도 가능

# ====== (C) 유틸 ======
def embed(texts: List[str]) -> np.ndarray:
    return encoder.encode(texts, normalize_embeddings=True)

def cosine(x, y):
    return float(util.cos_sim(np.asarray(x), np.asarray(y)))

@dataclass
class SlotResult:
    label: str
    score: float
    topk: List[Tuple[str, float]]

class SimilarityClassifier:
    def __init__(self, label_dict: Dict[str, List[str]], literal_boost: float = 0.15):
        """
        label_dict: {"라벨": ["별칭1","별칭2",...]}
        literal_boost: 본문에 별칭이 '문자 그대로' 등장하면 가산점
        """
        self.label_dict = label_dict
        self.literal_boost = literal_boost
        # 각 별칭을 미리 임베딩(오프라인/CPU 최적화)
        self.alias_texts = []
        self.alias_labels = []
        for lab, aliases in label_dict.items():
            for a in aliases:
                self.alias_texts.append(a)
                self.alias_labels.append(lab)
        self.alias_emb = embed(self.alias_texts)  # (N, dim)

    def predict(self, text: str, top_k: int = 3) -> SlotResult:
        t = text.strip()
        q = embed([t])[0]  # (dim,)
        # 별칭 임베딩과 유사도
        sims = (self.alias_emb @ q).tolist()  # cosine (정규화돼 있으므로 내적=코사인)
        # literal match 가산점
        low = t.lower()
        for i, alias in enumerate(self.alias_texts):
            if alias in t or alias.lower() in low:
                sims[i] += self.literal_boost

        # 별칭별 최고 점수 → 라벨별 최고 점수로 집계
        best_per_label: Dict[str, float] = {}
        for s, lab in zip(sims, self.alias_labels):
            if lab not in best_per_label or s > best_per_label[lab]:
                best_per_label[lab] = s

        # 정렬
        ranked = sorted(best_per_label.items(), key=lambda x: x[1], reverse=True)
        topk = ranked[:top_k]
        label, score = topk[0]
        return SlotResult(label=label, score=float(score), topk=topk)

# ====== (D) 에이전트 ======
class FactoryAIAgent:
    def __init__(self):
        self.part_clf   = SimilarityClassifier(FAULT_PARTS, literal_boost=0.20)
        self.defect_clf = SimilarityClassifier(DEFECT_TYPES, literal_boost=0.15)
        self.action_clf = SimilarityClassifier(ACTIONS, literal_boost=0.15)

    def predict(self, fault_text: str, top_k: int = 3):
        part   = self.part_clf.predict(fault_text, top_k=top_k)
        defect = self.defect_clf.predict(fault_text, top_k=top_k)
        action = self.action_clf.predict(fault_text, top_k=top_k)

        return {
            "input": fault_text,
            "고장 부품": {"pred": part.label, "score": round(part.score, 3), "topk": part.topk},
            "불량 유형": {"pred": defect.label, "score": round(defect.score, 3), "topk": defect.topk},
            "조치 내용": {"pred": action.label, "score": round(action.score, 3), "topk": action.topk},
        }

    # 현장 피드백으로 라벨/별칭을 즉시 보강(러닝 없이 업데이트)
    def add_alias(self, slot: str, label: str, alias: str):
        slot_map = {"part": self.part_clf, "defect": self.defect_clf, "action": self.action_clf}
        clf = slot_map[slot]
        clf.label_dict.setdefault(label, []).append(alias)
        clf.alias_texts.append(alias)
        clf.alias_labels.append(label)
        # 임베딩 1개만 추가
        emb = embed([alias])
        clf.alias_emb = np.vstack([clf.alias_emb, emb])


agent = FactoryAIAgent()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
samples = [
    "출하장 입구 에어 50A 유니온에서 에어리크 발생하여 재조임 후 해결함.",
    "컨베이어 모터에서 진동과 소음이 커져 베어링 교체 진행.",
    "실린더 로드 주변 오일 누설 발견, 패킹 마모로 판단되어 교체 조치.",
    "센서 오염으로 감지 불량 발생, 세척 후 정상.",
    "밸브 정렬불량으로 에어 흐름 불안정, 얼라인 재조정으로 해결."
]

for s in samples:
    print("문장:", s)
    print(agent.predict(s), "\n")


문장: 출하장 입구 에어 50A 유니온에서 에어리크 발생하여 재조임 후 해결함.
{'input': '출하장 입구 에어 50A 유니온에서 에어리크 발생하여 재조임 후 해결함.', '고장 부품': {'pred': '유니온', 'score': 0.648, 'topk': [('유니온', 0.6478485584259033), ('실린더', 0.5774869322776794), ('밸브', 0.5751441717147827)]}, '불량 유형': {'pred': '에어리크', 'score': 0.75, 'topk': [('에어리크', 0.7503632545471192), ('파손', 0.4513343572616577), ('오염', 0.42856985330581665)]}, '조치 내용': {'pred': '재조임', 'score': 0.669, 'topk': [('재조임', 0.6687984108924866), ('교체', 0.5756605863571167), ('수리', 0.5656445026397705)]}} 

문장: 컨베이어 모터에서 진동과 소음이 커져 베어링 교체 진행.
{'input': '컨베이어 모터에서 진동과 소음이 커져 베어링 교체 진행.', '고장 부품': {'pred': '베어링', 'score': 0.895, 'topk': [('베어링', 0.8950482726097106), ('컨베이어', 0.8378188133239746), ('실린더', 0.5829168558120728)]}, '불량 유형': {'pred': '전기불량', 'score': 0.442, 'topk': [('전기불량', 0.4418768286705017), ('진동', 0.43142232298851013), ('마모', 0.43057340383529663)]}, '조치 내용': {'pred': '수리', 'score': 0.619, 'topk': [('수리', 0.6192852258682251), ('교체', 0.6028673648834229), ('윤활', 0.530395865

In [ ]:
!pip -q install gradio
import gradio as gr
import pandas as pd

def ui_predict(text):
    out = agent.predict(text, top_k=3)
    # 보기 좋게 표 형태로
    rows = []
    for k in ["고장 부품", "불량 유형", "조치 내용"]:
        pred = out[k]["pred"]; score=out[k]["score"]
        cand = ", ".join([f"{l}({s:.2f})" for l, s in out[k]["topk"]])
        rows.append([k, pred, score, cand])
    df = pd.DataFrame(rows, columns=["항목","예측","점수","Top-3 후보(점수)"])
    return df, out

with gr.Blocks() as demo:
    gr.Markdown("## 공장 고장내용 → 자동 분류 ")
    inp = gr.Textbox(lines=4, label="고장 내용 입력")
    btn = gr.Button("분류")
    table = gr.Dataframe(headers=["항목","예측","점수","Top-3 후보(점수)"], wrap=True)
    #j = gr.JSON(label="원시 출력(JSON)")
    btn.click(ui_predict, inputs=inp, outputs=[table, j])

demo.launch(share=False)  # 내부망에서만 사용


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>